# 🎯 ЭТАП 1: ОПТИМИЗАЦИЯ MZA КЛАССИФИКАТОРА

## 📅 Дата: 26.10.2025
## 🎯 Цель: Достичь максимального качества разделения рыночных зон

### 📊 **Задачи:**
1. **Оптимизировать динамические веса** для разных уровней волатильности
2. **Настроить пороговые значения** для всех индикаторов MZA
3. **Протестировать на разных таймфреймах** - найти оптимальные параметры
4. **Создать систему автоматической настройки** параметров

### 🏆 **Ожидаемые результаты:**
- Максимальное качество разделения зон (Economic Value > 0.0005)
- Оптимизированные параметры для всех таймфреймов
- Система автоматической настройки параметров
- Готовность к оптимизации индикаторов


## 📦 ИМПОРТЫ И НАСТРОЙКИ


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройки визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Настройки pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("📦 Импорты загружены успешно")
print(f"📅 Дата: {datetime.now().strftime('%d.%m.%Y %H:%M')}")


## 🔧 ЗАГРУЗКА СИСТЕМЫ ОПТИМИЗАЦИИ


In [ ]:
# Импорт базовой системы оптимизации
import sys
sys.path.append('..')

from base_optimization_system import IndicatorOptimizationSystem

# Создание системы оптимизации с MZA классификатором
mza_system = IndicatorOptimizationSystem(classifier_type='mza')

print("✅ MZA система оптимизации загружена")
print(f"📊 Доступные таймфреймы: {list(mza_system.data.keys())}")
print(f"🔧 Классификатор: {mza_system.classifier_type}")
